In [1]:
from jax import jit, grad, vmap, random
from functools import partial
import jax.numpy as jnp
from utils_fs_v2 import nonlinear_DNN, linear_DNN, DNN
from anytree import NodeMixin
from jax.tree_util import tree_flatten, tree_unflatten, register_pytree_node, tree_structure

No GPU/TPU found, falling back to CPU. (Set TF_CPP_MIN_LOG_LEVEL=0 and rerun for more info.)


#### Defining useful parameters

In [2]:
# N_sf = 200 # single fidelity hidden width
# N_nl = 80 # multifidelity nonlinear network hidden width
N_sf = 4 # single fidelity hidden width
N_nl = 4 # multifidelity nonlinear network hidden width
sfnet_shape = [1, N_sf, N_sf, 2] # shape of the single fidelity network
nonlin_mfnet_shape = [3, N_nl, N_nl, 2] # shape of the multifidelity nonlinear network
lin_mfnet_shape = [2,  4, 2] # shape of the multifidelity linear network
init_nl, apply_nl, weight_nl = nonlinear_DNN(nonlin_mfnet_shape)
init_l, apply_l = linear_DNN(lin_mfnet_shape)
init_sf, apply_sf = DNN(sfnet_shape)

#### Defining the function for applying the multifidelity network

In [3]:
def apply_mf(self,params,pts,u_lf):
    # params = self.unravel_params # NOTE: This is probably wrong. I am not certain what format the two functions below are expecting
    u_nl = apply_nl(params[1], jnp.hstack([pts, u_lf]))
    u_l = apply_l(params[0], u_lf)
    u_local = u_l + u_nl
    # w = weight(pts)
    # u_local = w*(u_l + u_nl)
    return u_local

#### Defining the classes `SFDomain` and `MFDomain`

In [16]:
class RootUtilities:
    def __init__(self):
        self.levels = [] # list of what classes are on which levels of the domain tree

class SFDomain(NodeMixin,RootUtilities):
    def __init__(self,vertices,params_prev=[]):
        super(SFDomain,self).__init__()
        if len(params_prev) > 0:
            params = params_prev
        else:
            params_sf = init_sf(random.PRNGKey(1))
            params = (params_sf)
        self.params = params
        self.vertices = vertices
        self.tree_level_organizer(self) # add root to the level organizer


    def register_children(self):
        registration = []
        for child in self.children:
            registration.append(child.__repr__())
        return registration
    
    def tree_level_organizer(self,new_node): # TESTED
        """
        "tree_level_organizer" recognizes which level "new_node" is on and places "new_node"
        in the appropriate level in "self.levels".
        =================================================================================================
        INPUT:
        new_node:   A class that is being added to the tree
        """
        depth = new_node.depth # find the depth of the new node
        if depth >= len(self.levels): # if "new_node" is the first on a new level, add that level to "self.levels"
            self.levels.append([new_node])
        else: # if "new_node" is a member of an existing level, add it to that level of "self.levels"
            self.levels[depth].append(new_node)


    def __repr__(self):
        return "SFDomain(vertices={}, params={},children={})".format(self.vertices, "sf_params", self.register_children())

class MFDomain(NodeMixin):
    def __init__(self,vertices,params_prev=[],parent=None):
        if len(params_prev) > 0:
            params = params_prev
        else:
            params_nl = init_nl(random.PRNGKey(13))
            params_l = init_l(random.PRNGKey(12345))
            params = (params_l, params_nl)
        self.params = params
        self.vertices = vertices
        self.parent = parent
        self.root.tree_level_organizer(self) # add current node to the level organizer 

    
    def register_children(self):
        registration = []
        for child in self.children:
            registration.append(child.__repr__())
        return registration

    def __repr__(self):
        return "MFDomain(vertices={}, params={},children={})".format(self.vertices, "mf_params", self.register_children())

#### Registering `SFDomain` so that it can be jitted. I believe that it works

In [17]:
def return_leaves(v):
  leaves = [v.vertices,v.params]
  # leaves = (v.name,)
  for child in v.children:
    leaves.append(return_leaves(child))
    # leaves = leaves + (return_leaves(child),)
  return leaves

def special_flatten(v):
  leaves = [v.vertices,v.params]
  # leaves = (v.name,)
  for child in v.children:
    leaves.append(return_leaves(child))
    # leaves = leaves + (return_leaves(child),)
  aux_data = None
  return (leaves, aux_data)

def grow_tree(parent,tree):
  new_parent = MFDomain(tree[0],tree[1],parent=parent)
  for branch in tree[2:]:
    grow_tree(new_parent,branch)

def special_unflatten(aux_data, tree):
  root = SFDomain(tree[0],tree[1])
  for branch in tree[2:]:
    grow_tree(root,branch)
  return root

# Global registration
register_pytree_node(
    SFDomain,
    special_flatten,    # tell JAX what are the children nodes
    special_unflatten   # tell JAX how to pack back into a RegisteredSpecial
)

In [18]:
A = SFDomain([[0.0],[1.0]])
B = MFDomain([[0.0],[0.6]],parent = A)
C = MFDomain([[0.4],[1.0]],parent = A)
# D = MFDomain([[0.0],[0.3]],parent = B)
# E = MFDomain([[0.2],[0.5]],parent = B)
# F = MFDomain([[0.4],[0.7]],parent = C)
# G = MFDomain([[0.6],[1.0]],parent = C)

In [40]:
def is_interior(vertices,pt):
    return ((vertices[0] < pt) & (pt < vertices[1])).all()

def get_vertices(domain):
    return domain.vertices

@jit
def multifidelity_network(root,pt,max_level):
    u_pred = apply_sf(root.params,pt) # get the single fidelity approximation
    cache = jnp.empty(5)
    cache.at[0].set(root)
    # for idx in range(1,max_level+1):
    #     level = root.levels[idx]
    #     vertices = jnp.array([mfdomain.vertices for mfdomain in level])
    #     is_in = vmap(is_interior,(0,None))(vertices,pt)
    
    # u = root.levels[1][0]
    # print(u)
    # print(root)
    # print(root.depth)
    # for bool in is_in:
    #     if bool:
    #         u_lf = 
    #         go_deeper


    return u_pred

In [41]:
pts = jnp.linspace(0,1,21)
vmap(multifidelity_network,(None,0,None))(A,pts,1)

TypeError: Unexpected input type for array: <class '__main__.SFDomain'>

In [32]:
jnp.array(range(1,3))

Array([1, 2], dtype=int32)

#### The below example shows how to register a tree so that one can utilize the tree inside of a @jit function

We define a class called `Special` and a class called `SpecialLeaf`. The `Special` class serves as the root and the `SpecialLeaf` class will be used to create nodes on this tree. Below we create a function that will output the flattened and unflattened trees.

In [ ]:
def show_example(structured):
  flat, tree = tree_flatten(structured)
  unflattened = tree_unflatten(tree, flat)
  print(f"{structured=}\n  {flat=}\n  {tree=}\n  {unflattened=}")

In [137]:
class Special(NodeMixin):
  def __init__(self, name):
    self.name = name
  
  def register_children(self):
    registration = []
    # registration = ()
    for child in self.children:
      registration.append(child.__repr__())
      # registration = registration + (child.__repr__(),)
    return registration

  def __repr__(self):
    return "Special(name={}, children={})".format(self.name, self.register_children())

class SpecialLeaf(NodeMixin):
  def __init__(self, name, parent):
    self.name = name
    self.parent = parent

  def register_children(self):
    registration = []
    # registration = ()
    for child in self.children:
      registration.append(child.__repr__())
      # registration = registration + (child.__repr__(),)
    return registration

  def __repr__(self):
    return "SpecialLeaf(name={}, children={})".format(self.name,self.register_children())

In [158]:
A = Special(1)
B = SpecialLeaf(2,parent=A)
C = SpecialLeaf(3,parent=A)
D = SpecialLeaf(4,parent=B)

In [160]:
show_example(A)

structured=Special(name=1, children=["SpecialLeaf(name=2, children=['SpecialLeaf(name=4, children=[])'])", 'SpecialLeaf(name=3, children=[])'])
  flat=[Special(name=1, children=["SpecialLeaf(name=2, children=['SpecialLeaf(name=4, children=[])'])", 'SpecialLeaf(name=3, children=[])'])]
  tree=PyTreeDef(*)
  unflattened=Special(name=1, children=["SpecialLeaf(name=2, children=['SpecialLeaf(name=4, children=[])'])", 'SpecialLeaf(name=3, children=[])'])


In [180]:
class RegisteredSpecial(Special):
  def __repr__(self):
    return "Special(name={}, children={})".format(self.name,self.register_children())
  
def return_leaves(v):
  leaves = [v.name]
  # leaves = (v.name,)
  for child in v.children:
    leaves.append(return_leaves(child))
    # leaves = leaves + (return_leaves(child),)
  return leaves

def special_flatten(v):
  leaves = [v.name]
  # leaves = (v.name,)
  for child in v.children:
    leaves.append(return_leaves(child))
    # leaves = leaves + (return_leaves(child),)
  aux_data = None
  return (leaves, aux_data)

def grow_tree(parent,tree):
  new_parent = SpecialLeaf(tree[0],parent=parent)
  for branch in tree[1:]:
    grow_tree(new_parent,branch)

def special_unflatten(aux_data, tree):
  root = RegisteredSpecial(tree[0])
  for branch in tree[1:]:
    grow_tree(root,branch)
  return root

# Global registration
register_pytree_node(
    RegisteredSpecial,
    special_flatten,    # tell JAX what are the children nodes
    special_unflatten   # tell JAX how to pack back into a RegisteredSpecial
)


In [181]:
a = RegisteredSpecial(1)
b = SpecialLeaf(2,parent=a)
c = SpecialLeaf(3,parent=a)
d = SpecialLeaf(4,parent=b)

In [184]:
show_example(a)

structured=Special(name=1, children=["SpecialLeaf(name=2, children=['SpecialLeaf(name=4, children=[])'])", 'SpecialLeaf(name=3, children=[])'])
  flat=[1, 2, 4, 3]
  tree=PyTreeDef(CustomNode(RegisteredSpecial[None], [*, [*, [*]], [*]]))
  unflattened=Special(name=1, children=["SpecialLeaf(name=2, children=['SpecialLeaf(name=4, children=[])'])", 'SpecialLeaf(name=3, children=[])'])


In [187]:
@jit
def print_node(root,temp):
    return root.name

In [190]:
temp = jnp.linspace(0,1,4)
vmap(print_node,(None,0))(a,temp)

Array([1, 1, 1, 1], dtype=int32, weak_type=True)

In [191]:
a.name = 2

In [192]:
temp = jnp.linspace(0,1,4)
vmap(print_node,(None,0))(a,temp)

Array([2, 2, 2, 2], dtype=int32, weak_type=True)

In [27]:
@partial(jit,static_argnums=(0,1))
def foo(i,f):
    f = foo3
    return f(i)

def foo2(i):
    return 2*i

def foo3(i):
    return 3*i

In [28]:
foo(6,foo2)

Array(18, dtype=int32, weak_type=True)

In [32]:
def f(x,x_copy,y):
  if x_copy < 3:
    return 3. * x * y
  else:
    return -4 * x **2 * y

f = grad(jit(f, static_argnums=(1,)),0)

In [33]:
print(f(4.,4.,1.))

-32.0


In [34]:
print(f(5.,5.,1.))

-40.0


In [20]:
import numpy as np

v = np.arange(10)
v[:1]
v[1]

1

In [40]:
@partial(jit, static_argnums = (1,))
def foo(x,x_copy):
    vals = jnp.zeros(3)
    if x_copy[0] > 0:
        vals[0] = x[0]
    else:
        vals[0] = -x[0]

    if x_copy[1] > 0:
        vals[1] = x[1]
    else:
        vals[1] = -x[1]

    if x_copy[2] > 0:
        vals[2] = x[2]
    else:
        vals[2] = -x[2]
        
    return vals

In [41]:
# pts = jnp.array([2.,3.,4.])
# pts_copy = pts.copy()
# foo(pts,pts_copy)

pts = jnp.array([2., 3., 4.])
pts_copy = pts.copy()
foo(tuple(pts), pts_copy)

ValueError: Non-hashable static arguments are not supported. An error occurred during a call to 'foo' while trying to hash an object of type <class 'jaxlib.xla_extension.ArrayImpl'>, [2. 3. 4.]. The error was:
TypeError: unhashable type: 'ArrayImpl'


In [64]:
from jax import vmap
from jax.numpy import zeros, where

# @partial(jit, static_argnums = (1,))
def foo(x, x_copy):
    vals = zeros(3)
    vals = where(x > 0, x, -x)
    # print(x)
    print(x_copy)
    return vals

pts = jnp.array([2., -3., 4.])
pts_copy = pts.copy()
# vmap(foo)(pts, pts_copy)
foo(pts, pts_copy)

[ 2. -3.  4.]


Array([2., 3., 4.], dtype=float32)

In [66]:
from jax import lax

In [74]:
val = jnp.array([1.])
lax.cond(1 > 0, lambda: 1, lambda: 2)

Array(1, dtype=int32, weak_type=True)